# Preprocessing

This file makes folders and processes the TIMIT dataset to filterbanks

In [1]:
import scipy.io.wavfile
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import os
import pandas as pd

In [3]:
# Change to change output folder
OUTPUT_PATH = Path('./AUDIO_FILTER_BANKS')

TRAIN_PATH = Path('./TRAIN_WAV')
TEST_PATH = Path('./TEST_WAV')

# All of the sub directories, such as TRAIN_PATH/DR1
TRAIN_PATHS = [x for x in TRAIN_PATH.iterdir()]
TEST_PATHS = [x for x in TEST_PATH.iterdir()]

In [5]:
# All of the different accents/people talking
TRAIN_ACCENTS = [[x for x in xs.iterdir()] for xs in TRAIN_PATHS]
TEST_ACCENTS = [[x for x in xs.iterdir()] for xs in TEST_PATHS]

In [9]:
pat = r'^([A-Z0-9]+)\.[A-Z]+$'

def get_files(pathss):
    """
        Returns all of the files in a folder and
        all of the file-groupings in a path.
        For example if we have a folder with multiple files
        such as path/test.png, path/test.txt etc.
        Then this will return path/test
    """
    FILES = []
    for paths in pathss:
        for xs in paths:
            for x in xs.iterdir():
                TRAIN_FILES.append(x)
        #TRAIN_FILES.append([[x for x in xs.iterdir()] for xs in paths])
    FILES2 = []
    matches = []

    for x in TRAIN_FILES:
        # I might be able to use x.stem insead of this
        # Path has .match for regex matching also. Might be quicker and better
        match = re.search(pat, x.name)
        if match == None or match.group(1) in matches:
            continue
        FILES2.append(x.with_suffix(''))
        matches.append(match.group(1))
    return TRAIN_FILES, TRAIN_FILES_2

# TRAIN_FILES here is the files while TRAIN_FILES_2 are the file groupings
(TRAIN_FILES_ALL, TRAIN_FILES_GROUPINGS) = get_files(TRAIN_ACCENTS)
(TEST_FILES_ALL, TEST_FILES_GROUPINGS) = get_files(TEST_ACCENTS)
print(TRAIN_FILES_2[:5])
print(TRAIN_FILES[:5])
#print(TEST_FILES_2[:5])

[PosixPath('TRAIN_WAV/DR7/MMDG0/SX160'), PosixPath('TRAIN_WAV/DR7/MMDG0/SA1'), PosixPath('TRAIN_WAV/DR7/MMDG0/SA2'), PosixPath('TRAIN_WAV/DR7/MMDG0/SI2035'), PosixPath('TRAIN_WAV/DR7/MMDG0/SX340')]
[PosixPath('TRAIN_WAV/DR7/MMDG0/SX160.PHN'), PosixPath('TRAIN_WAV/DR7/MMDG0/SA1.TXT'), PosixPath('TRAIN_WAV/DR7/MMDG0/SA1.WAV.wav'), PosixPath('TRAIN_WAV/DR7/MMDG0/SA2.PHN'), PosixPath('TRAIN_WAV/DR7/MMDG0/SX70.WAV.wav')]


In [ ]:
path = Path(OUTPUT_PATH)
path.mkdir(parents=True, exist_ok=True)

In [ ]:
#TRAIN_FILES_2 are each of the peoples voices. Need to iterate over
def make_directories(files, index = "TRAIN_WAV"):
    #print(files[0])
    for i, file in enumerate(files):
        print(x.parts)
        index = file.parts.index(index)
        t = Path(OUTPUT_PATH).joinpath(*file.parts[index + 1:])
        #print(t)
        t.mkdir(parents=True, exist_ok=True)

In [11]:
TRAIN_FILES_PHONES = []
TEST_FILES_PHONES = []
phones = [
    "aa",
    "ae",
    "ah",
    "ah",
    "ay",
    "aw",
    "b",
    "ch",
    "d",
    "dh",
    "dx",
    "eh",
    "er",
    "ey",
    "f",
    "g",
    "hh",
    "ih",
    "k",
    "iy",
    "jh",
    "l",
    "m",
    "n",
    "ng",
    "oy",
    "ow",
    "r",
    "s",
    "sh",
    "t",
    "th",
    "uw",
    "uh",
    "p",
    "v",
    "w",
    "y",
    "z",
    "sil"
]

#OUTPUT_PATH = Path(OUTPUT_PATH / "TRAIN")

for i in phones:
    TRAIN_FILES_PHONES.append(OUTPUT_PATH / "TRAIN" / i)
    
#OUTPUT_PATH = Path(OUTPUT_PATH / "TEST")

for i in phones:
    TEST_FILES_PHONES.append(OUTPUT_PATH / "TEST" / i)
    
print(TRAIN_FILES_PHONES[:5])
print(TRAIN_FILES_PHONES[:5])

for d in TRAIN_FILES_PHONES:
    d.mkdir(parents=True, exist_ok=True)
for d in TEST_FILES_PHONES:
    d.mkdir(parents=True, exist_ok=True)
    
#make_directories(TRAIN_FILES_3, "AUDIO_FIX_FILTER_BANKS_DELTA_DELTA_FOLDERS")

[PosixPath('AUDIO_FILTER_BANKS/TRAIN/aa'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ae'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ah'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ah'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ay')]
[PosixPath('AUDIO_FILTER_BANKS/TRAIN/aa'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ae'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ah'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ah'), PosixPath('AUDIO_FILTER_BANKS/TRAIN/ay')]


In [25]:

def get_phn(file, i):
    """
        Gets the phone of file file at time i
    """
    phns = pd.read_csv(file.with_suffix("").with_suffix('.PHN'), sep = " " , header=None)
    phn = phns[2][0]
    for idx, x in enumerate(phns[0]):
        #print(i, idx, x, phns[2][idx])
        if i >= x:
            phn = phns[2][idx]
            continue
        return phn
        break
    else:
        return phn
            

In [30]:
# phn = pd.read_csv(TRAIN_FILES_2[0].with_suffix('.PHN'), sep = " " , header=None)
#print(phn[1])
i = 4
print(TRAIN_FILES[i])
get_phn(TRAIN_FILES[i], 10280)

TRAIN_WAV/DR7/MMDG0/SX70.WAV.wav


'q'